## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-15-06-56-25 +0000,nypost,Firefighters make progress against fast-moving...,https://nypost.com/2025/08/15/us-news/firefigh...
1,2025-08-15-06-52-07 +0000,bbc,Suspected French spy arrested in alleged Mali ...,https://www.bbc.com/news/articles/cj9w7dg3pr4o...
2,2025-08-15-06-48-37 +0000,nyt,"As Trump Pushes International Students Away, A...",https://www.nytimes.com/2025/08/14/business/us...
3,2025-08-15-06-39-00 +0000,wsj,Dollar Falls Ahead of Trump-Putin Meeting,https://www.wsj.com/finance/currencies/yen-mos...
4,2025-08-15-06-08-50 +0000,nypost,FDNY swimmer saves person clinging to bottom o...,https://nypost.com/2025/08/15/us-news/fdny-swi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2301/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,58
32,putin,25
164,will,13
188,year,11
149,ukraine,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
53,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...,141
18,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...,127
211,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,124
39,2025-08-15-02-38-36 +0000,nypost,Anchorage prepares for Trump-Putin summit as s...,https://nypost.com/2025/08/14/us-news/anchorag...,123
258,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,121


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
53,141,2025-08-15-01-00-00 +0000,wsj,"President Trump, by meeting Vladimir Putin on ...",https://www.wsj.com/politics/national-security...
18,69,2025-08-15-04-27-14 +0000,nypost,AG Pam Bondi declares DC ‘will not remain a sa...,https://nypost.com/2025/08/15/us-news/ag-pam-b...
95,51,2025-08-14-22-26-00 +0000,wsj,California Democrats are planning their own re...,https://www.wsj.com/politics/policy/gavin-news...
80,43,2025-08-14-23-11-41 +0000,startribune,"New details, charges emerge as Vance Boelter i...",https://www.startribune.com/hennepin-county-gr...
156,37,2025-08-14-19-03-21 +0000,cbc,"WATCH | Zelenskyy, Starmer meet ahead of Trump...",https://www.cbc.ca/player/play/9.6867280?cmp=rss
235,35,2025-08-14-14-14-14 +0000,cbc,"With flight attendant strike looming, union ac...",https://www.cbc.ca/news/business/air-canada-pa...
7,34,2025-08-15-05-45-33 +0000,nypost,Former North Carolina brewery owner charged fo...,https://nypost.com/2025/08/15/us-news/former-o...
197,32,2025-08-14-16-23-09 +0000,nypost,"Toronto film festival reverses course, will ‘e...",https://nypost.com/2025/08/14/world-news/tiff-...
155,32,2025-08-14-19-04-00 +0000,wsj,U.N. Report Finds Widespread Killing by Syrian...,https://www.wsj.com/world/middle-east/u-n-repo...
278,30,2025-08-14-10-00-00 +0000,nypost,Ukraine’s kids forced to study online — or und...,https://nypost.com/2025/08/14/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
